# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
K. Lee  ->  K. Lee  |  ['K. Lee']
G. Mattia  ->  G. Mattia  |  ['G. Mattia']
S. Li  ->  S. Li  |  ['S. Li']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']


F. Walter  ->  F. Walter  |  ['F. Walter']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 101 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2411.18735


extracting tarball to tmp_2411.18735...

 done.
Retrieving document from  https://arxiv.org/e-print/2411.18999


extracting tarball to tmp_2411.18999...

...

extracting tarball to tmp_2411.19945...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2411.19945/iso_cen.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Found 129 bibliographic references in tmp_2411.19945/iso_cen.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.19126-b31b1b.svg)](https://arxiv.org/abs/2411.19126) | **Spectral and magnetic properties of the jet base in NGC 315**  |
|| L. Ricci, et al. -- incl., <mark>G. Mattia</mark> |
|*Appeared on*| *2024-12-03*|
|*Comments*| *16 pages, 19 figures, accepted for publication in A&A*|
|**Abstract**|            The dynamic of relativistic jets in the inner parsec regions is deeply affected by the nature of the magnetic fields. The level of magnetization of the plasma, as well as the geometry of these fields on compact scales, have not yet been fully constrained. In this paper we employ multi-frequency and multi-epoch very long baseline interferometry observations of the nearby radio galaxy NGC 315. We aim to derive insights into the magnetic field properties on sub-parsec and parsec scales by examining observational signatures such as the spectral index, synchrotron turnover frequency, and brightness temperature profiles. This analysis is performed by considering the properties of the jet acceleration and collimation zone, which can be probed thanks to the source vicinity, as well as the inner part of the jet conical region. We observe remarkably steep values for the spectral index on sub-parsec scales ($\alpha \sim -2$, $S_\nu \propto \nu^\alpha$) which flatten around $\alpha \sim -0.8$ on parsec scales. We suggest that the observed steep values may result from particles being accelerated via diffusive shock acceleration mechanisms in magnetized plasma and subsequently experiencing cooling through synchrotron losses. The brightness temperature of the 43 GHz cores indicates a dominance of the magnetic energy at the jet base, while the cores at progressively lower frequencies reveal a gradual transition towards equipartition. Based on the spectral index and brightness temperature along the incoming jet, and by employing theoretical models, we derive that the magnetic field strength has a close-to-linear dependence with distance going from parsec scales up to the jet apex. Overall, our findings are consistent with a toroidal-dominated magnetic field on all the analyzed scales.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.19489-b31b1b.svg)](https://arxiv.org/abs/2411.19489) | **ATOMS: ALMA Three-millimeter Observations of Massive Star-forming regions -- XIX. The origin of SiO emission**  |
|| R. Liu, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2024-12-03*|
|*Comments*| *23 pages, 14 figures*|
|**Abstract**|            The production of silicon monoxide (SiO) can be considered as a fingerprint of shock interaction. In this work, we use high-sensitivity observations of the SiO (2-1) and H$^{13}$CO$^{+}$ (1-0) emission to investigate the broad and narrow SiO emission toward 146 massive star-forming regions in the ATOMS survey. We detected SiO emission in 136 regions and distinguished broad and narrow components across the extension of 118 sources (including 58 UC $H_{II}$ regions) with an average angular resolution of 2.5$^{\prime}$$^{\prime}$. The derived SiO luminosity ($L_{SiO}$) across the whole sample shows that the majority of $L_{SiO}$ (above 66$\%$) can be attributed to broad SiO, indicating its association with strong outflows. The comparison of the ALMA SiO images with the filamentary skeletons identified from H$^{13}$CO$^{+}$ and in the infrared data (at 4.5, 8, and 24 $mu$m), further confirms that most SiO emission originates from outflows. However, note that for nine sources in our sample, the observed SiO emission may be generated by expanding UC $H_{II}$ regions. There is a moderate positive correlation between the bolometric luminosity ($L_{bol}$) and $L_{SiO}$ for both components (narrow and broad). The UC $H_{II}$ sources show a weaker positive correlation between $L_{bol}$ and $L_{SiO}$ and higher $L_{SiO}$ compared to the sources without UC $H_{II}$ regions. These results imply that the SiO emission from UC $H_{II}$ sources might be affected by UV-photochemistry induced by UC $H_{II}$ regions.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.19686-b31b1b.svg)](https://arxiv.org/abs/2411.19686) | **MIRI Deep Imaging Survey (MIDIS) of the Hubble Ultra Deep Field**  |
|| G. Östlin, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-12-03*|
|*Comments*| *submitted to A&A on July 30, 2024*|
|**Abstract**|            The recently launched James Webb Space Telescope (JWST) is opening new observing windows on the distant universe. Among JWST's instruments, the Mid Infrared Instrument (MIRI) offers the unique capability of imaging observations at wavelengths $\lambda > 5\mu$m. This enables unique access to the rest frame near infra-red (NIR, $\lambda \ge 1$\mum) emission from galaxies at redshifts $z>4$ and the visual ($\lambda \gtrsim 5000$Å) rest frame for $z>9$. We here report on the guaranteed time observations (GTO) from the MIRI European Consortium, of the Hubble Ultra Deep Field (HUDF), forming the MIRI Deep Imaging Survey (MIDIS), consisting of an on source integration time of $\sim41$ hours in the MIRI/F560W (5.6 $\mu$m) filter. To our knowledge, this constitutes the longest single filter exposure obtained with JWST of an extragalactic field as yet.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.19916-b31b1b.svg)](https://arxiv.org/abs/2411.19916) | **A gravitational wave detectable candidate Type Ia supernova progenitor**  |
|| E. T. Chickles, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2024-12-03*|
|*Comments*| *40 pages, 7 figures, 2 tables*|
|**Abstract**|            Type Ia supernovae, critical for studying cosmic expansion, arise from thermonuclear explosions of white dwarfs, but their precise progenitor pathways remain unclear. Growing evidence supports the ``double-degenerate'' scenario, where two white dwarfs interact. The absence of other companion types capable of explaining the observed Ia rate, along with observations of hyper-velocity white dwarfs interpreted as surviving companions of such systems provide compelling evidence in favor of this scenario. Upcoming millihertz gravitational wave observatories like the Laser Interferometer Space Antenna (LISA) are expected to detect thousands of double-degenerate systems, though the most compact known candidate Ia progenitors produce only marginally detectable gravitational wave signals. Here, we report observations of ATLAS J1138-5139, a binary white dwarf system with an orbital period of 28 minutes. Our analysis reveals a 1 solar mass carbon-oxygen white dwarf accreting from a helium-core white dwarf. Given its mass, the accreting carbon-oxygen white dwarf is poised to trigger a typical-luminosity Type Ia supernova within a few million years, or to evolve into a stably mass-transferring AM CVn system. ATLAS J1138-5139 provides a rare opportunity to calibrate binary evolution models by directly comparing observed orbital parameters and mass transfer rates closer to merger than any previously identified candidate Type Ia progenitor. Its compact orbit ensures detectability by LISA, demonstrating the potential of millihertz gravitational wave observatories to reveal a population of Type Ia progenitors on a Galactic scale, paving the way for multi-messenger studies offering insights into the origins of these cosmologically significant explosions.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.19945-b31b1b.svg)](https://arxiv.org/abs/2411.19945) | **The hot circumgalactic medium in the eROSITA All-Sky Survey III. Star-forming and quiescent galaxies**  |
|| Y. Zhang, et al. -- incl., <mark>A. Pillepich</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-12-03*|
|*Comments*| *16 pages, 12 figures, accepted for publication in A&A*|
|**Abstract**|            The circumgalactic medium (CGM), as the gas repository for star formation, might contain the answer to the mysterious galaxy quenching and bimodal galaxy population origin. We measured the X-ray emission of the hot CGM around star-forming and quiescent galaxies. We detect extended X-ray emission from the hot CGM around star-forming galaxies with $\log(M_*/M_\odot)>11.0$ and quiescent galaxies with $\log(M_*/M_\odot)>10.5$, extending out to $R_{\rm 500c}$. $L_{\rm X, CGM}$ of star-forming galaxies with median stellar masses $\log(M_{\rm *,med}/M_\odot) = 10.7, 11.1, 11.3$ are approximately $0.8\,, 2.3\,, 4.0 \times 10^{40}\,\rm erg/s$, while for quiescent galaxies with $\log(M_{\rm *,med}/M_\odot) = 10.8, 11.1, 11.4$, they are $1.1\,, 6.2\,, 30 \times 10^{40}\,\rm erg/s$. Notably, quiescent galaxies with $\log(M_{\rm *,med}/M_\odot) > 11.0$ exhibit brighter hot CGM than their star-forming counterparts. In halo mass bins, we detect similar X-ray emission around star-forming and quiescent galaxies with $\log(M_{\rm 200m}/M_\odot) > 12.5$, suggesting that galaxies in the same mass dark matter halos host equally bright hot CGM. We emphasize the observed $L_{\rm X, CGM} - M_{\rm 500c}$ relations of star-forming and quiescent galaxies are sensitive to the stellar-to-halo mass relation (SHMR). A comparison with cosmological hydrodynamical simulations (EAGLE, TNG100, and SIMBA) reveals varying degrees of agreement, contingent on the simulation and the specific stellar or halo mass ranges considered. Either selected in stellar mass or halo mass, the star-forming galaxies do not host brighter stacked X-ray emission from the hot CGM than their quiescent counterparts at the same mass range. The result provides useful constraints on the extent of feedback's impacts as a mechanism for quenching star formation as implemented in current cosmological simulations.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.19651-b31b1b.svg)](https://arxiv.org/abs/2411.19651) | **Ice inventory towards the protostar Ced 110 IRS4 observed with the James Webb Space Telescope. Results from the ERS Ice Age program**  |
|| W. R. M. Rocha, et al. -- incl., <mark>G. Perotti</mark> |
|*Appeared on*| *2024-12-03*|
|*Comments*| *33 pages, 19 Figures. Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            This work focuses on the ice features toward the binary protostellar system Ced 110 IRS 4A and 4B, and observed with JWST as part of the Early Release Science Ice Age collaboration. We aim to explore the JWST observations of the binary protostellar system Ced~110~IRS4A and IRS4B to unveil and quantify the ice inventories toward these sources. We compare the ice abundances with those found for the same molecular cloud. The analysis is performed by fitting or comparing laboratory infrared spectra of ices to the observations. Spectral fits are carried out with the ENIIGMA fitting tool that searches for the best fit. For Ced~110~IRS4B, we detected the major ice species H$_2$O, CO, CO$_2$ and NH$_3$. All species are found in a mixture except for CO and CO$_2$, which have both mixed and pure ice components. In the case of Ced~110~IRS4A, we detected the same major species as in Ced~110~IRS4B, as well as the following minor species CH$_4$, SO$_2$, CH$_3$OH, OCN$^-$, NH$_4^+$ and HCOOH. Tentative detection of N$_2$O ice (7.75~$\mu$m), forsterite dust (11.2~$\mu$m) and CH$_3^+$ gas emission (7.18~$\mu$m) in the primary source are also presented. Compared with the two lines of sight toward background stars in the Chameleon I molecular cloud, the protostar has similar ice abundances, except in the case of the ions that are higher in IRS4A. The clearest differences are the absence of the 7.2 and 7.4~$\mu$m absorption features due to HCOO$^-$ and icy complex organic molecules in IRS4A and evidence of thermal processing in both IRS4A and IRS4B as probed by the CO$_2$ ice features. We conclude that the binary protostellar system Ced~110~IRS4A and IRS4B has a large inventory of icy species. The similar ice abundances in comparison to the starless regions in the same molecular cloud suggest that the chemical conditions of the protostar were set at earlier stages in the molecular cloud.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.18735-b31b1b.svg)](https://arxiv.org/abs/2411.18735) | **What Can DKIST/DL-NIRSP Tell Us About Quiet-Sun Magnetism?**  |
|| <mark>J. Liu</mark>, X. Sun, P. W. Schuck, S. A. Jaeggli |
|*Appeared on*| *2024-12-03*|
|*Comments*| *29 pages, 22 figures. Accepted for publication in ApJ*|
|**Abstract**|            Quiet-Sun regions cover most of the Sun's surface; its magnetic fields contribute significantly to the solar chromospheric and coronal heating. However, characterizing the magnetic fields of the quiet Sun is challenging due to their weak polarization signal. The 4-m \textit{Daniel K. Inouye Solar Telescope} (\textit{DKIST}) is expected to improve our understanding of the quiet-Sun magnetism. In this paper, we assess the diagnostic capability of the Diffraction-Limited Near Infrared Spectropolarimeter (DL-NIRSP) instrument on \textit{DKIST} on the energy transport processes in the quiet-Sun photosphere. To this end, we synthesize high-resolution, high-cadence Stokes profiles of the \ion{Fe}{1} 630~nm lines using a realistic magnetohydrodynamic simulation, degrade them to emulate the \textit{DKIST}/DL-NIRSP observations, and subsequently infer the vector magnetic and velocity fields. For the assessment, we first verify that a widely used flow-tracking algorithm, Differential Affine Velocity Estimator for Vector Magnetograms, works well for estimating the large-scale ($> 200$ km) photospheric velocity fields with these high-resolution data. We then examine how the accuracy of inferred velocity depends on the temporal resolution. Finally, we investigate the reliability of the Poynting flux estimate and its dependence on the model assumptions. The results suggest that the unsigned Poynting flux, estimated with existing schemes, can account for about $71.4\%$ and $52.6\%$ of the reference ground truth at $\log \tau =0.0$ and $\log \tau = -1$. However, the net Poynting flux tends to be significantly underestimated. The error mainly arises from the underestimated contribution of the horizontal motion. We discuss the implications on \textit{DKIST} observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.18999-b31b1b.svg)](https://arxiv.org/abs/2411.18999) | **Rapid Rotation of Polarization Orientations in PSR B1919+21's Single Pulses: Implications On Pulsar's Magnetospheric Dynamics**  |
|| S. Cao, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2024-12-03*|
|*Comments*| *14 pages, 7 figures. Submitted to ApJL*|
|**Abstract**|            We analyse and model rapid rotations of polarization orientations in PSR B1919+21's single pulses, based on FAST observation data. In more than one third of B1919+21's single pulses, polarization angle (PA) is found to rotate quasi-monotonically with pulse longitude, by over 180 or even 360 degrees. Most of these quasi-monotonic PA curves have negative slopes with respect to pulse longitude. Oscillations of circular polarization fraction accompany these PA rotations. This rapid rotation could be induced by quick change of phase lag between normal wave modes within an individual pulse. We propose a phenomenological model to successfully reproduce the observed polarization rotations in single pulses, and calculate phase lags in a dipolar magnetic field of an aligned rotating pulsar, with a dispersion relation of orthogonal wave modes in strongly magnetized plasma. The weak frequency dependence of observed polarization rotation requires the radio emission height to be low, approximately 10 times the neutron star radius.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2411.19126/./Images/Core_shift_VLBA.png', 'tmp_2411.19126/./Images/Turnover_frequency_profile.png', 'tmp_2411.19126/./Images/VLBA_SM_profiles_5_perc_3.png']
copying  tmp_2411.19126/./Images/Core_shift_VLBA.png to _build/html/
copying  tmp_2411.19126/./Images/Turnover_frequency_profile.png to _build/html/
copying  tmp_2411.19126/./Images/VLBA_SM_profiles_5_perc_3.png to _build/html/
exported in  _build/html/2411.19126.md
    + _build/html/tmp_2411.19126/./Images/Core_shift_VLBA.png
    + _build/html/tmp_2411.19126/./Images/Turnover_frequency_profile.png
    + _build/html/tmp_2411.19126/./Images/VLBA_SM_profiles_5_perc_3.png
found figures ['tmp_2411.19489/./Figure/Figure10/LM_Lsio_N_new_final.png', 'tmp_2411.19489/./Figure/Figure3/I08470-4243.moment04.png', 'tmp_2411.19489/./Figure/Figure3/I11332-6258.moment010.png', 'tmp_2411.19489/./Figure/Figure3/I11298-6155.moment09.png', 'tmp_2411.19489/./Figure/Figure3/I12326-6245.moment013.png', 'tmp_2411.19489/./Figure/Figure5

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\jr}[1]{\textcolor{orange}{JR: #1}}$
$\newcommand{\gm}{\color{blue}}$
$\newcommand{\mk}[1]{\textcolor{violet}{MK: #1}}$
$\newcommand$</div>



<div id="title">

# Spectral and magnetic properties of the jet base in NGC 315

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.19126-b31b1b.svg)](https://arxiv.org/abs/2411.19126)<mark>Appeared on: 2024-12-03</mark> -  _16 pages, 19 figures, accepted for publication in A&A_

</div>
<div id="authors">

L. Ricci, et al. -- incl., <mark>G. Mattia</mark>

</div>
<div id="abstract">

**Abstract:**            The dynamic of relativistic jets in the inner parsec regions is deeply affected by the nature of the magnetic fields. The level of magnetization of the plasma, as well as the geometry of these fields on compact scales, have not yet been fully constrained. In this paper we employ multi-frequency and multi-epoch very long baseline interferometry observations of the nearby radio galaxy NGC 315. We aim to derive insights into the magnetic field properties on sub-parsec and parsec scales by examining observational signatures such as the spectral index, synchrotron turnover frequency, and brightness temperature profiles. This analysis is performed by considering the properties of the jet acceleration and collimation zone, which can be probed thanks to the source vicinity, as well as the inner part of the jet conical region. We observe remarkably steep values for the spectral index on sub-parsec scales ($\alpha \sim -2$, $S_\nu \propto \nu^\alpha$) which flatten around $\alpha \sim -0.8$ on parsec scales. We suggest that the observed steep values may result from particles being accelerated via diffusive shock acceleration mechanisms in magnetized plasma and subsequently experiencing cooling through synchrotron losses. The brightness temperature of the 43 GHz cores indicates a dominance of the magnetic energy at the jet base, while the cores at progressively lower frequencies reveal a gradual transition towards equipartition. Based on the spectral index and brightness temperature along the incoming jet, and by employing theoretical models, we derive that the magnetic field strength has a close-to-linear dependence with distance going from parsec scales up to the jet apex. Overall, our findings are consistent with a toroidal-dominated magnetic field on all the analyzed scales.         

</div>

<div id="div_fig1">

<img src="tmp_2411.19126/./Images/Core_shift_VLBA.png" alt="Fig2" width="100%"/>

**Figure 2. -** Core position as a function of frequency for the two different data sets: i) the multi-frequency VLBA data set (orange points); ii) the multi-frequency and multi-epoch data set presented in [Boccardi, Perucho and Casadio (2021)]()(blue points). The blue line represents the best-fit values presented in [Boccardi, Perucho and Casadio (2021)](), the orange line traces the best-fit performed in this paper by employing all the data, and the purple line highlights the best-fit curve obtained using only the data points at 8, 15, 22, and 43 GHz. (*fig:core_shift*)

</div>
<div id="div_fig2">

<img src="tmp_2411.19126/./Images/Turnover_frequency_profile.png" alt="Fig3" width="100%"/>

**Figure 3. -** Turnover frequency along the ridgeline as a function of distance from the 43 GHz core. The multiple data points at the same distance are from the different possible core-shift configurations. The orange points represent the upper boundary, while the green ones the lower boundary. The turnover frequency decreases from $10   \mathrm{GHz} \lesssim \nu_\mathrm{br} \lesssim 35   \mathrm{GHz}$ in the 43 GHz core, down to $\sim 6   \mathrm{GHz}$ at $\sim 0.7   \mathrm{pc}$. (*fig:turnover_evolution*)

</div>
<div id="div_fig3">

<img src="tmp_2411.19126/./Images/VLBA_SM_profiles_5_perc_3.png" alt="Fig7" width="100%"/>

**Figure 7. -** Average spectral index as a function of distance from the 43 GHz core for different pairs of frequencies. In addition to the VLBA data set here presented, we re-present for comparison the 22-43 GHz spectral index values for the two epochs presented by [Ricci, Boccardi and Nokhrina (2022)](). At high frequencies, remarkably steep spectral index values down to $\alpha \sim -2$ are observed within one parsec from the core, corresponding to ${\sim}10^4   R_\mathrm{S}$. Downstream, a convergence towards flatter values $\alpha \sim -0.8$ is observed at all frequencies. The black vertical lines highlight the jet break point, as proposed in  ([Boccardi, Perucho and Casadio 2021]()) , while the horizontal dashed line is set at $\alpha = -1$ as reference. (*fig:alpha_profiles*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.19126"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# ATOMS: ALMA Three-millimeter Observations of Massive Star-forming regions – $\uppercase$$\expandafter{\romannumeral19}$. The origin of SiO emission

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.19489-b31b1b.svg)](https://arxiv.org/abs/2411.19489)<mark>Appeared on: 2024-12-03</mark> -  _23 pages, 14 figures_

</div>
<div id="authors">

R. Liu, et al. -- incl., <mark>S. Li</mark>

</div>
<div id="abstract">

**Abstract:** The production of silicon monoxide (SiO) can be considered as a fingerprint of shock interaction.In this work, we use high-sensitivity observations of the SiO (2-1) and H $^{13}$ CO $^{+}$ (1-0) emission to investigate the broad and narrow SiO emission toward 146 massive star-forming regions in the ATOMS survey. We detected SiO emission in 136 regions and distinguished broad and narrow components across the extension of 118 sources (including 58 UC H ii regions) with an average angular resolution of 2.5 $^{\prime}^{\prime}$ .The derived SiO luminosity ( $L_\textup{SiO}$ ) across the whole sample shows that the majority of $L_\textup{SiO}$ (above 66 $\%$ ) can be attributed to broad SiO, indicating its association with strong outflows.The comparison of the ALMA SiO images with the filamentary skeletons identified from H $^{13}$ CO $^{+}$ and in the infrared data (at 4.5, 8, and 24 $\upmu$ m), further confirms that most SiO emission originates from outflows. However, note that for nine sources in our sample, the observed SiO emission may be generated by expanding UC H ii regions. There is a moderate positive correlation between the bolometric luminosity ( $L_\textup{bol}$ ) and $L_\textup{SiO}$ for both components (narrow and broad). The UC H ii sources show a weaker positive correlation between $L_\textup{bol}$ and $L_\textup{SiO}$ and higher $L_\textup{SiO}$ compared to the sources without UC H ii regions. These results imply that the SiO emission from UC H ii sources might be affected by UV-photochemistry induced by UC H ii regions.

</div>

<div id="div_fig1">

<img src="tmp_2411.19489/./Figure/Figure10/LM_Lsio_N_new_final.png" alt="Fig10" width="100%"/>

**Figure 10. -** _ Upper panels_: The SiO luminosity ($L_\textup{sio}$) versus  bolometric luminosity ($L_\textup{bol}$).
The filled and empty gray stars present broad and narrow SiO components in _A groups_ without UC Hii regions, and the filled and empty gray circles depict broad and narrow SiO
components in _B groups_ without UC Hii regions.
The filled and empty red stars show broad and narrow SiO components in _A groups_ hosting UC Hii regions, while the filled and empty red circles display broad and narrow SiO components in _B groups_ with UC Hii regions.
The filled green rectangles show the low-mass stars  (2022MNRAS.512.5214D, [Jiménez-Serra, et. al 2011](https://ui.adsabs.harvard.edu/abs/2011ApJ...739...80J), [Spezzano, et. al 2020](https://ui.adsabs.harvard.edu/abs/2020A&A...640A..74S), [Santiago-García, et. al 2009](https://ui.adsabs.harvard.edu/abs/2009A&A...495..169S), [ and Lee 2020](https://ui.adsabs.harvard.edu/abs/2020A&ARv..28....1L)) .
In the upper left panel, the gray line shows a linear fit of log $(L_\textrm{sio} / L_{\sun})$$= (0.64\pm0.07)$ log $(L_\textrm{bol} / L_{\sun})$ - $6.98\pm0.32$ obtained for sources without UC Hii regions, while the red line shows the linear fit log $(L_\textrm{sio} / L_{\sun})$$ = (0.56\pm0.11)$ log $(L_\textrm{bol} / L_{\sun})$ - $6.81\pm0.58$ obtained for sources hosting UC Hii sources.
In the upper right panel, the gray line displays the linear fit
log $(L_\textrm{sio} / L_{\sun})$$ = (0.65\pm0.08)$ log $(L_\textrm{bol} / L_{\sun})$ - $7.30\pm0.33$ derived for sources without UC Hii regions, while the red line is used to show the linear fit
log $(L_\textrm{sio} / L_{\sun})$$ = (0.40\pm0.13)$ log $(L_\textrm{bol} / L_{\sun})$ - $6.31\pm0.68$ inferred for sources hosting UC Hii regions.
_ Lower panels_: $L_\textup{sio}$ vs. $L_\textup{bol}/M$. No apparent correlation is seen either for the broad or for the narrow components. The symbols are the same as in the upper panels. (*fig10*)

</div>
<div id="div_fig2">

<img src="tmp_2411.19489/./Figure/Figure3/I08470-4243.moment04.png" alt="Fig4.1" width="25%"/><img src="tmp_2411.19489/./Figure/Figure3/I11332-6258.moment010.png" alt="Fig4.2" width="25%"/><img src="tmp_2411.19489/./Figure/Figure3/I11298-6155.moment09.png" alt="Fig4.3" width="25%"/><img src="tmp_2411.19489/./Figure/Figure3/I12326-6245.moment013.png" alt="Fig4.4" width="25%"/>

**Figure 4. -** Four representative sources imaged with ALMA within the ATOMS program. The background corresponds to the SiO (2-1) integrated intensity maps. The black contours show the 3 mm continuum emission detected with ALMA, and contours are from 5$\sigma$ to the peak values in steps of 10$\sigma$.
The bold gray lines represent the filament skeletons identified using H$^{13}$CO$^+$ as reported by zhou2022atoms. The green dashed rectangle is the area of the SiO emission maps where the SiO line profiles have been decomposed into different velocity components. The orange and blue rectangles, with the broadest and narrowest SiO line widths, respectively, indicate the locations where SiO and H$^{13}$CO$^+$ have been extracted.
The field of view (FOV) is 72$^{\prime}^{\prime}$ corresponding with the FOV of the ALMA observations. All images have been primary-beam corrected. The source name and integrated velocity ranges (in km s$^{-1}$) are shown on the upper left and right corners, respectively. The beam size is reported in the lower left corner. The same images are provided for all sources within the supplementary material. (*fig3*)

</div>
<div id="div_fig3">

<img src="tmp_2411.19489/./Figure/Figure5/SiO_velocity_final_report2.png" alt="Fig6.1" width="50%"/><img src="tmp_2411.19489/./Figure/Figure5/SiO_Width_final_report2.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** The histogram shows the velocity offset compared to the systemic velocity and line width distribution of the broad and narrow components of the SiO emission for the entire sample. The open blue histograms correspond to the sources without outflow activity, and the orange-filled histograms represent the sources exhibiting outflow activity. The outflow sources can be found in Table \ref{tab:TableA1}. (*fig5*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.19489"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mum}{{\mum}}$
$\newcommand{\mjy}{{\muJy}}$
$\newcommand{\lya}{{Ly\alpha}}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\pa}{Pa \alpha}$
$\newcommand{\brb}{Br\beta}$
$\newcommand{\brg}{Br\gamma}$
$\newcommand{\brd}{Br\delta}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\hg}{H\gamma}$
$\newcommand{\hd}{H\delta}$
$\newcommand{\oii}{[O {\sc ii}]}$
$\newcommand{\oiii}{[O {\sc iii}]}$
$\newcommand{\nii}{[N {\sc ii}]}$
$\newcommand{\hi}{H {\sc i}}$
$\newcommand{\hii}{H {\sc ii}}$
$\newcommand{\zsun}{Z_\odot}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\msunyr}{M_\odot/yr}$
$\newcommand{\sfrkpc}{M_\odot/yr/kpc^2}$
$\newcommand{\lfuv}{L_{\rm FUV}}$
$\newcommand{\lsun}{L_\odot}$
$\newcommand{\bull}{\bullet}$
$\newcommand{\fesc}{f_{esc, Ly\alpha}}$
$\newcommand{\fesclyc}{f_{esc}}$
$\newcommand{\hiz}{high-z}$
$\newcommand{\z}{z}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\ergc}{{erg/s/cm^2/Å }}$
$\newcommand{\ergl}{{erg/s/cm^2}}$
$\newcommand{\htwo}{{H_2}}$
$\newcommand{\fet}{{[Fe{\sc ii}]}}$
$\newcommand{\flam}{{f_{\lambda}}}$
$\newcommand{\lamrest}{\lambda_{\rm rest}}$
$\newcommand{\lamobs}{\lambda_{\rm obs}}$</div>



<div id="title">

#  MIRI Deep Imaging Survey (MIDIS) of the Hubble Ultra Deep Field$\thanks{Based on results from the MIRI European Consortium Guaranteed Time Observations, program 1283}$ : Project description and early results for the galaxy population detected at 5.6 $\mum$ 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.19686-b31b1b.svg)](https://arxiv.org/abs/2411.19686)<mark>Appeared on: 2024-12-03</mark> -  _submitted to A&A on July 30, 2024_

</div>
<div id="authors">

G. Östlin, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** The recently launched James Webb Space Telescope (JWST) is opening new observing windows on the distant universe. Among JWST's instruments, the Mid Infrared Instrument (MIRI) offers the unique capability of imaging observations at wavelengths $\lambda > 5\mu$ m. This enables unique access to the rest frame near infra-red (NIR, $\lambda \ge 1$ $\mum$ ) emission from galaxies at redshifts $z>4$ and the visual ( $\lambda \gtrsim 5000$ Å) rest frame for $z>9$ . We here report on the guaranteed time observations (GTO) from the MIRI European Consortium, of the Hubble Ultra Deep Field (HUDF), forming the MIRI Deep Imaging Survey (MIDIS), consisting of an on source integration time of $\sim41$ hours in the MIRI/F560W (5.6 $\mu$ m) filter.   To our knowledge, this constitutes the longest single filter exposure obtained with JWST of an extragalactic field as yet. The HUDF is one of the most observed  extragalactic fields, with extensive multi-wavelength coverage, where (before  JWST) galaxies up to $z\sim 7$ have been  confirmed, and at $z>10$ suggested, from HST photometry. We aim to characterise the galaxy population in HUDF at 5.6 $\mu$ m, enabling studies such as: the rest frame NIR morphologies for galaxies at $z\lesssim4.6$ ,  probing mature stellar populations and emission lines  in $z>6$ sources, intrinsically red and dusty galaxies, and active galactic nuclei (AGN) and their host galaxies at intermediate redshifts. We have reduced the MIRI data using the $*JWST*$ pipeline, augmented by in-house custom scripts. We measure the noise characteristics of the resulting image. Galaxy photometry has been obtained, and photometric redshifts have been estimated for sources with available multi wavelength photometry (and compared to spectroscopic redshifts when available). Over the deepest part of our image  the 5 $\sigma$ point source limit is 28.65 mag AB (12.6 nJy), $\sim0.35$ mag better than predicted by the JWST exposure time calculator.   We find $\sim2500$ sources, the overwhelming majority of which are distant galaxies, but note that spurious sources likely remain at faint magnitudes due to imperfect cosmic ray rejection in the JWST pipeline.   More than 500 galaxies with available spectroscopic redshifts, up to $z\approx11$ have been identified, the majority of which are at $z<6$ .   More than 1000 galaxies have reliable photometric redshift estimates, of which $\sim25$ are  at $6<z<12$ .   The point spread function in the F560W filter has a FWHM of $\approx0.2\arcsec$ (corresponding to $1.4$ kpc at $z=4$ ), allowing the near infrared rest frame   morphologies  for the first time to be resolved up to $z\sim4$ . As expected, the  light distributions are smoother than at shorter wavelength, and trace the stellar mass distributions.    Moreover, $>100$ objects with very red  NIRCam vs MIRI (3.6--5.6 $\mum$ $>1$ ) colours have been found, indicating dusty or old stellar populations at high redshifts. We conclude that MIDIS  surpasses pre-flight expectations and that deep MIRI imaging has a great potential for characterizing the galaxy population from cosmic noon to dawn.

</div>

<div id="div_fig1">

<img src="tmp_2411.19686/./Figures/midis_colordist_F356W-F560W_zcol.png" alt="Fig14" width="100%"/>

**Figure 14. -** F560W magnitude vs NIRCAM/F356W--F560W color. Only sources with a F560W (F356W) magnitude uncertainty of 0.2 (0.5) or lower are included. The red circle indicates a confirmed MERO currently under investigation (Jermann et al. in prep). The colors indicate photometric redshift of the sources (gray points do not have a valid redshift estimate). (*color-mag*)

</div>
<div id="div_fig2">

<img src="tmp_2411.19686/./Figures/hudf_mds_kron_err+hist.png" alt="Fig4" width="100%"/>

**Figure 4. -** 
   Top: F560W number counts for Kron magnitudes.
   Bottom: F560W Kron magnitudes vs photometric uncertainty. The derived $5\sigma$ point source limiting magnitude is indicated by the dashed vertical line.  (*f560err+hist*)

</div>
<div id="div_fig3">

<img src="tmp_2411.19686/./Figures/exptime_map_2col.png" alt="Fig9" width="100%"/>

**Figure 9. -** Left: The F560W image with linear intensity scaling. Colored patches show regions where we have estimated the noise (see Table \ref{depthtable}) with different depths: the deepest area (A), where obs 1-2, 4-6 overlap (yellow). The deep area (B), where obs 4 does not overlap fully with the others (green). Outside these areas, there is coverage at less depth, notably the NE extension which only has observations from obs 4. The area with at least 7h of combined integration is denoted C (purple). The orientation is in the detector plane (x,y). Right: Exposure time map. (*image-exposuremap*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.19686"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\mn}{{Mon. Not. R. Astron. Soc.}}$
$\newcommand{\mnras}{\mn}$
$\newcommand{\aj}{{"Astron. J."}}$
$\newcommand{\apj}{{Astrophys. J.}}$
$\newcommand{\apjl}{{Astrophys. J. Lett.}}$
$\newcommand{\apjs}{{Astrophys. J. Supp.}}$
$\newcommand{\apss}{{Astrophys. and Space Sc.}}$
$\newcommand{\aaps}{{Astron. Astrophys. Supp.}}$
$\newcommand{\aap}{{Astron. Astrophys.}}$
$\newcommand{\nat}{{Nature}}$
$\newcommand{\pasj}{{PASJ}}$
$\newcommand{\prd}{{Phys. Rev. D}}$
$\newcommand{\prl}{{Phys. Rev. Lett.}}$
$\newcommand{\pasp}{{Pub. Ast. Soc. Pac.}}$
$\newcommand{\procspie}{Proc. SPIE}$
$\newcommand{\ssr}{Space Science Reviews}$
$\newcommand{\araa}{Annual Review of Astronomy and Astrophysics}$
$\newcommand{\thetable}{\arabic{table}}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand\url{#1}$
$\newcommand{\urlprefix}{URL }$
$\newcommand\url{#1}$
$\newcommand{\urlprefix}{URL }$
$\newcommand\url{#1}$
$\newcommand{\urlprefix}{URL }$</div>



<div id="title">

# A gravitational wave detectable candidate Type Ia supernova progenitor

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.19916-b31b1b.svg)](https://arxiv.org/abs/2411.19916)<mark>Appeared on: 2024-12-03</mark> -  _40 pages, 7 figures, 2 tables_

</div>
<div id="authors">

E. T. Chickles, et al. -- incl., <mark>K. El-Badry</mark>

</div>
<div id="abstract">

**Abstract:** Type Ia supernovae, critical for studying cosmic expansion \citemain{1998AJ....116.1009R} $^,$ \citemain{1998ApJ...507...46S} , arise from thermonuclear explosions of white dwarfs \citemain{1960ApJ...132..565H} , but their precise progenitor pathways remain unclear. Growing evidence supports the "double-degenerate" scenario, where two white dwarfs interact \citemain{1984ApJS...54..335I} $^,$ \citemain{1984ApJ...277..355W} . The absence of other companion types capable of explaining the observed Ia rate \citemain{1984ApJ...277..355W} , along with observations of hyper-velocity white dwarfs interpreted as surviving companions of such systems \citemain{2018ApJ...865...15S} $^,$ \citemain{2023OJAp....6E..28E} provide compelling evidence in favor of this scenario. Upcoming millihertz gravitational wave observatories like the Laser Interferometer Space Antenna (LISA) \citemain{2017arXiv170200786A} are expected to detect thousands of double-degenerate systems, though the most compact known candidate Ia progenitors produce only marginally detectable gravitational wave signals. Here, we report observations of ATLAS J1138-5139, a binary white dwarf system with an orbital period of just 28 minutes. Our analysis reveals a 1 solar mass carbon-oxygen white dwarf accreting from a helium-core white dwarf. Given its mass, the accreting carbon-oxygen white dwarf is poised to trigger a typical-luminosity Type Ia supernova within a few million years, or to evolve into a stably mass-transferring AM CVn system. sudo apt install texlive-font-utilsATLAS J1138-5139 provides a rare opportunity to calibrate binary evolution models by directly comparing observed orbital parameters and mass transfer rates closer to merger than any previously identified candidate Type Ia progenitor. Its compact orbit ensures detectability by LISA, demonstrating the potential of millihertz gravitational wave observatories to reveal a population of Type Ia progenitors on a Galactic scale, paving the way for multi-messenger studies offering insights into the origins of these cosmologically significant explosions.

</div>

<div id="div_fig1">

<img src="tmp_2411.19916/./Figures/Fig_LC.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Light curve of ATLAS J1138-5139.****a)** The binned $u_sg_sr_s$ ULTRACAM light curve of ATLAS J1138-5139, phase-folded on the 27.68-min orbital period. The light curve exhibits sinusoidal variations, with maxima at phases $0.25$ and $0.75$, due to an ellipsoidally deformed secondary star. At phase $0.75$, a heated accretion feature contributes additional flux, resulting in the appearance of unequal maxima. **b)** The asymmetric primary eclipse in the *ugr* ULTRACAM light curve, indicating the presence of a luminous hot spot on the outer edge of an accretion disk surrounding the primary star. **c)** The binned and phase-folded light curve of the object from ATLAS. We were able to discover the object because of its periodic behavior. **d)** The binned and phase-folded TESS light curve. (*fig:LC*)

</div>
<div id="div_fig2">

<img src="tmp_2411.19916/./Figures/dust_edenhofer2023.png" alt="Fig5" width="100%"/>

**Figure 5. -** Interstellar dust extinction maps illustrating the extinction along the line of sight to ATLAS J1138-5139 at three distances. The first column corresponds to the system's observed distance $d=557$ pc, the second column shows the extinction at $d-\sigma_{d,\text{lower}}=541$ pc, and the third column represents $d+\sigma_{d,\text{upper}}=576$ pc, where $\sigma_d$ is the uncertainty in the distance measurement. The location of the system is marked with a red "X" in all panels. The values of $E(B-V)$ are negligible across all distances. (*fig:dust*)

</div>
<div id="div_fig3">

<img src="tmp_2411.19916/./Figures/lightcurve_model.png" alt="Fig6" width="100%"/>

**Figure 6. -** Three color ULTRACAM light curve from March 20, 2023, overlaid with the best-fit toy \texttt{LCURVE} model (solid black lines). The model includes contributions from an accretion disk and a hot spot where the accretion stream intersects the outer disk, which were needed to achieve an acceptable fit to the lightcurve. The best fit model gives a near edge-on inclination (88.6 degrees), though we chose not to include this model as part of our joint fit, due to the large number of model-dependent free parameters involved, as compared to our more simple and robust approach. (*fig:LCURVE*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.19916"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\jc}[1]{\textcolor{purple}{JC: #1}}$
$\newcommand{\gab}[1]{\textcolor{orange}{GP: #1}}$
$\newcommand{\yi}[1]{\textcolor{brown}{#1}}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# The hot circumgalactic medium in the eROSITA All-Sky Survey: III. Star-forming and quiescent galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.19945-b31b1b.svg)](https://arxiv.org/abs/2411.19945)<mark>Appeared on: 2024-12-03</mark> -  _16 pages, 12 figures, accepted for publication in A&A_

</div>
<div id="authors">

Y. Zhang, et al. -- incl., <mark>A. Pillepich</mark>, <mark>X. Zhang</mark>

</div>
<div id="abstract">

**Abstract:**            The circumgalactic medium (CGM), as the gas repository for star formation, might contain the answer to the mysterious galaxy quenching and bimodal galaxy population origin. We measured the X-ray emission of the hot CGM around star-forming and quiescent galaxies. We detect extended X-ray emission from the hot CGM around star-forming galaxies with $\log(M_*/M_\odot)>11.0$ and quiescent galaxies with $\log(M_*/M_\odot)>10.5$, extending out to $R_{\rm 500c}$. $L_{\rm X, CGM}$ of star-forming galaxies with median stellar masses $\log(M_{\rm *,med}/M_\odot) = 10.7, 11.1, 11.3$ are approximately $0.8\,, 2.3\,, 4.0 \times 10^{40}\,\rm erg/s$, while for quiescent galaxies with $\log(M_{\rm *,med}/M_\odot) = 10.8, 11.1, 11.4$, they are $1.1\,, 6.2\,, 30 \times 10^{40}\,\rm erg/s$. Notably, quiescent galaxies with $\log(M_{\rm *,med}/M_\odot) > 11.0$ exhibit brighter hot CGM than their star-forming counterparts. In halo mass bins, we detect similar X-ray emission around star-forming and quiescent galaxies with $\log(M_{\rm 200m}/M_\odot) > 12.5$, suggesting that galaxies in the same mass dark matter halos host equally bright hot CGM. We emphasize the observed $L_{\rm X, CGM} - M_{\rm 500c}$ relations of star-forming and quiescent galaxies are sensitive to the stellar-to-halo mass relation (SHMR). A comparison with cosmological hydrodynamical simulations (EAGLE, TNG100, and SIMBA) reveals varying degrees of agreement, contingent on the simulation and the specific stellar or halo mass ranges considered. Either selected in stellar mass or halo mass, the star-forming galaxies do not host brighter stacked X-ray emission from the hot CGM than their quiescent counterparts at the same mass range. The result provides useful constraints on the extent of feedback's impacts as a mechanism for quenching star formation as implemented in current cosmological simulations.         

</div>

<div id="div_fig1">

<img src="tmp_2411.19945/./fig/profile/profile_halo_12p0_agnxrb_cgm.png" alt="Fig10.1" width="25%"/><img src="tmp_2411.19945/./fig/profile/profile_halo_12p5_agnxrb_cgm.png" alt="Fig10.2" width="25%"/><img src="tmp_2411.19945/./fig/profile/profile_halo_13p0_agnxrb_cgm.png" alt="Fig10.3" width="25%"/><img src="tmp_2411.19945/./fig/lxm/lx_CEN_halo_cgm_sfqu_r500.png" alt="Fig10.4" width="25%"/>

**Figure 10. -** X-ray surface brightness profiles of the hot CGM of the central star-forming ($S_{\rm X, CGM, SF}$, blue) and quiescent ($S_{\rm X, CGM, QU}$, red) galaxies with $M_{\rm 200m}\in 12.0-12.5$, $12.5-13.0$ and $13.0-13.5$(top left, top right and bottem left). The vertical dash-dotted lines denote the average $R_{\rm 500c}$ and $R_{\rm vir}$ of stacked galaxies. The median redshift ($\bar{z}_{\rm spec}$), median $M_{\rm 200m}$ and median $M_*$ are denoted. Hot CGM X-ray luminosity of central star-forming galaxies ($L_{\rm X,CGM, SF}$, blue) and quiescent galaxies ($L_{\rm X,CGM, QU}$, red) within $R_{\rm 500c}$ in $0.5-2$ keV as a function of $M_{\rm 500c}$, compared to the $L_{\rm X,CGM}-M_{\rm 500c}$ scaling relation from [Zhang, Comparat and Ponti (2024)]()(bottom right).  (*Fig_profile_cenhalo*)

</div>
<div id="div_fig2">

<img src="tmp_2411.19945/./fig/discuss/SHMR_compare_sf.png" alt="Fig2.1" width="50%"/><img src="tmp_2411.19945/./fig/discuss/SHMR_compare_qu.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Comparison of $M_{\rm 500c}-M*$ relations of central star-forming (top) and quiescent (bottom) galaxies in catalog [ and Tinker (2021)]() and [Yang and Mo (2007)](), and EAGLE, TNG100 and SIMBA simulations. The shadow area denotes the 16--84\% scatter of the relation. [Yang and Mo (2007)]() only includes galaxy groups with $\log(M_{\rm 500c})>11.5$ and causes the turnover of the quiescent SHMR at low mass end. (*Fig_shmr_yang*)

</div>
<div id="div_fig3">

<img src="tmp_2411.19945/./fig/cat/SFQU_MSMH.png" alt="Fig1" width="100%"/>

**Figure 1. -** The median $M_{\rm 200m}$ and median $M_*$ of galaxies with the $16-84\%$ scatter for each mass bin in the CEN$_{\rm halo,SF}$(blue band), CEN$_{\rm halo,QU}$(red band), CEN$_{\rm SF}$(green crosses), and CEN$_{\rm QU}$(orange cross) samples. (*Fig_SHMR*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.19945"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

263  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
